In [15]:
import os
import re
import csv
import operator
import collections
import numpy as np
from collections import Counter


In [16]:
# split word to ( , )

train_dir = 'trend_data1'
emails = [os.path.join(train_dir,f) for f in os.listdir(train_dir)]

all_words = []
for email in emails:
    with open(email, encoding = 'ISO-8859-1') as m:
        a = []
        for i,line in enumerate(m):
            if i != 1:
                words_ = line
                pattern3 = re.compile("[^\w\d]+")
                words_ = pattern3.sub(' ',words_)
                words = words_.split()
                for j in range(len(words)):
                    a.append(words[j])
    all_words.append(a)
print ('size of letter :', len(all_words))

size of letter : 21083


In [17]:
import numpy as np

np.save('all_words.npy', all_words)

In [18]:
cut_programs = np.load('all_words.npy')

In [19]:
# generate dictionary

word_dict = dict()

In [20]:
def add_word_dict(w):
    if not w in word_dict:
        word_dict[w] = 1
    else:
        word_dict[w] += 1

In [21]:
for letter in cut_programs:
    for w in letter:
        add_word_dict(w)

In [22]:
import operator

word_dict = sorted(word_dict.items(), key=operator.itemgetter(1), reverse=True)
print ('size of word :', len(word_dict))

size of word : 71742


In [25]:
# choose word by frequence 10 < f

VOC_SIZE = 5000
VOC_START = 50

voc_dict = word_dict[VOC_START:VOC_START+VOC_SIZE]
print(voc_dict[:10])
np.save('keyword_voc_dict.npy', voc_dict)

[('com', 3725), ('all', 3515), ('DIV', 3478), ('s', 3465), ('The', 3423), ('OF', 3407), ('20', 3377), ('ä', 3192), ('any', 3136), ('an', 2965)]


In [26]:
voc_dict = np.load('keyword_voc_dict.npy')

In [11]:
# word filter

example_doc = []

for letter in cut_programs:
    example_line = ''
    for w in letter:
        if w in voc_dict:
            example_line += w+' '
        
    example_doc.append(example_line)

print( example_doc[0] )

name Sani Abacha 2Cthe former first also wife former late head state Federal Republic Nigeria Gen Abacha whose sudden death came 8th June Since death 2CI been into present civilian administration been physical torture security agents country My son Mohammed under before federal high court Nigeria an he did commit he has been released since based certain conditions he allowed move around any where has been own town only also been placed under government watch As widow so lost confidence anybody within country You must heard over media reports internet various huge sums deposited husband different security firms abroad some companies give up their secrets disclosed lodged there many out right In fact total sum discovered Government so far tune 700 dollars And they make poor life 2EI got contacts through personal research 2Cand out decided reach through medium 2EI give more information regard soon reply repose great confidence hence due security network placed day day affairs cannot affor

### word2vec

In [12]:
# import modules & set up logging
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

C:\Users\nems\Anaconda3\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [13]:
# train word2vec on the two sentences
model = gensim.models.Word2Vec(example_doc, min_count=1, size=200, hs=1, negative=0)

2017-10-29 06:38:22,129 : INFO : collecting all words and their counts
2017-10-29 06:38:22,131 : WARNING : Each 'sentences' item should be a list of words (usually unicode strings). First item here is instead plain <class 'str'>.
2017-10-29 06:38:22,133 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-10-29 06:38:22,577 : INFO : PROGRESS: at sentence #10000, processed 4525456 words, keeping 85 word types
2017-10-29 06:38:22,633 : INFO : PROGRESS: at sentence #20000, processed 5026202 words, keeping 86 word types
2017-10-29 06:38:22,636 : INFO : collected 86 word types from a corpus of 5041719 raw words and 21083 sentences
2017-10-29 06:38:22,638 : INFO : Loading a fresh vocabulary
2017-10-29 06:38:22,640 : INFO : min_count=1 retains 86 unique words (100% of original 86, drops 0)
2017-10-29 06:38:22,642 : INFO : min_count=1 leaves 5041719 word corpus (100% of original 5041719, drops 0)
2017-10-29 06:38:22,644 : INFO : deleting the raw counts dictionary of 

In [14]:
model.save("word2vec.sav")

2017-10-29 06:50:45,965 : INFO : saving Word2Vec object under word2vec.sav, separately None
2017-10-29 06:50:45,967 : INFO : not storing attribute syn0norm
2017-10-29 06:50:45,970 : INFO : not storing attribute cum_table
2017-10-29 06:50:45,989 : INFO : saved word2vec.sav


### tfidf

In [ ]:
import math
import nltk
from textblob import TextBlob as tb

nltk.download('punkt')

def tf(word, blob):
    return blob.words.count(word) / len(blob.words)

def n_containing(word, bloblist):
    return sum(1 for blob in bloblist if word in blob)

def idf(word, bloblist):
    return math.log(len(bloblist) / (1 + n_containing(word, bloblist)))

def tfidf(word, blob, bloblist):
    return tf(word, blob) * idf(word, bloblist)

In [ ]:
bloblist = []
for i in range(len(example_doc)):
    blob = tb(example_doc[i])
    bloblist.append(blob)
print (len(bloblist))

In [ ]:
kw_n = 10

key_list = []
for i, blob in enumerate(bloblist):
    scores = {word: tfidf(word, blob, bloblist) for word in blob.words}
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    tmp = ""
    for word, score in sorted_words[ :kw_n]:
        tmp += word + ' '
    key_list.append(tmp)

In [ ]:
letter_score = []
for letter in key_list:
    if len(letter) == 0:
        pass
    else:
        score = model.score(letter.split())
        letter_score.append(score.reshape(1, score.shape[0]))

> how to get keyword of fraud ???

In [ ]:
# Cosine Similarity

from scipy import spatial

result = 1 - spatial.distance.cosine(letter_score[0], letter_score[1])
print (result)